<a href="https://colab.research.google.com/github/AlpinDale/misc-scripts/blob/main/Aphrodite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the Aphrodite Engine Colab!

If you're on mobile, please tab on the play button below. If you're not, you can safely skip it and go to the next cell.

The default model is Mythalion-13B, but you can either type in your model name, or select one of the defaults in the dropdown.

If you're running a non-quantized model, please set the Quantization to `None` instead. AWQ quantization is also available, but you will need Colab Pro with better GPUs.


In [ ]:
#@title <b>v-- Tap this if you play on Mobile</b> { display-mode: "form" }
%%html
<b>Press play on the music player to keep the tab alive, then start KoboldAI below (You can ignore this step if you used run all and are on PC)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
#@title <b>v-- Currently only GPTQ quantization is supported. For AWQ, please upgrade to Colab Pro</b>
#@markdown Currently only GPTQ quantization is supported. For AWQ, please upgrade to Colab Pro.
#@markdown You can enter a custom model as well (for models bigger than 13B GPTQ, you will need Colab Pro).

Model = "TheBloke/Mythalion-13B-GPTQ" #@param ["TheBloke/Mythalion-13B-GPTQ", "TheBloke/MythoMax-L2-13B-GPTQ", "TheBloke/Pygmalion-2-13B", "TheBloke/MLewd-L2-Chat-13B-GPTQ"]{allow-input: true}
Quantization = "gptq" #@param ['gptq', "None"]
!pip uninstall aphrodite-engine -y
!wget -O aphrodite_engine-0.3.4-cp310-cp310-manylinux1_x86_64.whl http://0x0.st/HWnW.whl && pip install aphrodite_engine-0.3.4-cp310-cp310-manylinux1_x86_64.whl
!wget -c https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!nohup ./cloudflared-linux-amd64 tunnel --url http://127.0.0.1:2242 &
!sleep 10
!cat nohup.out
!python -m aphrodite.endpoints.api_server_kobold --model $Model --dtype float16 --host 127.0.0.1 --quantization gptq --gpu-memory-utilization 0.88 &
